In [46]:
import scipy.io
import numpy as np
import scipy as sp
import cupy as cp
import matplotlib.pyplot as plt
import os
import time
from SWE_utils_cupy_standalone import *
from scipy.signal import firwin, butter, buttord, freqz

from arrus.ops.us4r import *
from arrus.ops.imaging import *
from arrus.metadata import *
from arrus.devices.probe import *
from arrus.devices.us4r import Us4RDTO
from arrus.utils.imaging import *

In [47]:
print(f"Device count: {cp.cuda.runtime.getDeviceCount()}")
device_props = cp.cuda.runtime.getDeviceProperties(0)

print(f"Device: {device_props['name']} (cc {device_props['major']}.{device_props['minor']})")
print(f"GPU clock frequency: {device_props['clockRate']/1e3} MHz")
print(f"SM Count: {device_props['multiProcessorCount']}")
print("Available memory: ")
print(f"- global memory: {device_props['totalGlobalMem']/2**20} MiB")
print(f"- shared memory per thread block: {device_props['sharedMemPerBlock']} B")
print(f"- constant memory: {device_props['totalConstMem']} B")

Device count: 1
Device: b'GeForce RTX 3060 Laptop GPU' (cc 8.6)
GPU clock frequency: 1425.0 MHz
SM Count: 30
Available memory: 
- global memory: 5946.5625 MiB
- shared memory per thread block: 49152 B
- constant memory: 65536 B


# Script parameters
Configure the reconstruction parameters of individual steps of the algorithm

In [48]:
# General settings
dataset_id = 340
directory  = '../Datasets'

# Constants
c              = 1540.0
probe_elements = 128
probe_pitch    = 0.298e-3
fs             = 65e6

# Sequence parameters
pwi_txFreq   = 4.4e6
pwi_nCycles  = 2
pwi_txAngles = [-4.0, 0.0, 4.0]
pwi_txPri    = 100e-6
pwi_fri      = 2* pwi_txPri

# RF Filter
rf_filter_band        = [4e6, 7e6]    # Desired pass band, Hz
rf_filter_trans_width = 1e6           # Width of transition from pass band to stop band, Hz
rf_filter_numtaps     = 236           # Size of the FIR filter.

# Post down conversion IQ filtering
demod_filter_cutoff = 0.5 * 4.4e6       # Desired cutoff frequency, Hz
demod_filter_trans_width = 0.5 * 4.4e6  # Width of transition from pass band to stop band, Hz
demod_filter_numtaps = 128              # Size of the FIR filter.

# Beamforming
px_size = 0.2  # in [mm]
x_grid = np.arange(-20, 20, px_size) * 1e-3
z_grid = np.arange(0, 50, px_size)   * 1e-3

# Shear wave detection
swd_mode              = 'kasai'
swd_zGate_length      = 4 
swd_ensemble_length   = 4

# Input parameters
df_sws_range = [0.5, 4.0];
df_f_range   = [40.0, 1000.0];
df_k_range   = 0.9;

# SWS estimation
swse_interp_factor = 5;
swse_interp_order  = 2
swse_d             = 14;
swse_frames        = [0, 99];
swse_SWV_range     = [0.5, 4.0];
swse_x_range       = [[0, 420], [0, 420]]
swse_z_clip        = [5, 10]

# Post-processing
median_filter_size = 5


## Load the dataset

In [49]:
# Load a dataset
sid = 'id' + str(dataset_id)
# Search for the file
for root, dirs, files in os.walk(directory):
    for file in files:
        if sid in file:
            file_path = os.path.join(root, file)
            data = sp.io.loadmat(file_path)

data["rf_data"].shape
rf = data["rf_data"]
rf = rf[np.newaxis, ...]
Nframes = rf.shape[1]
rf.shape

(1, 150, 4224, 128)

# Beamforming

### Design filters

In [50]:
## Function to plot the filter response
def plot_FIR_response(fs, taps, w, h):
    "Utility function to plot response functions"
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 9))
    ax0.plot(range(len(taps)), taps)
    ax0.set_xlabel("Tap")
    ax0.set_ylabel("Value")
    
    ax1.plot(0.5*fs*w/np.pi, 20*np.log10(np.abs(h)))
    ax1.set_ylim(-80, 5)
    ax1.set_xlim(0, 0.5*fs)
    ax1.grid(True)
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Gain (dB)')
    
    ax2.plot(0.5*fs*w/np.pi, np.angle(h))
    ax2.set_xlim(0, 0.5*fs)
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Phase (rad)")

## Design a band-pass FIR filter for RF filtering of raw channel data
band = rf_filter_band                # Desired pass band, Hz
trans_width = rf_filter_trans_width  # Width of transition from pass band to stop band, Hz
numtaps = rf_filter_numtaps          # Size of the FIR filter.

edges = [0, band[0] - trans_width, band[0], band[1], band[1] + trans_width, 0.5*fs]
rf_fir_taps = signal.remez(numtaps, edges, [0, 1, 0], Hz=fs)
if(0):
    w, h = signal.freqz(rf_fir_taps, [1], worN=2000)
    plot_FIR_response(fs, rf_fir_taps, w, h)

## Design a low-pass FIR filter for filtering of down-conversion products
# Specify the filter parameters    
cutoff = demod_filter_cutoff            # Desired cutoff frequency, Hz
trans_width = demod_filter_trans_width  # Width of transition from pass band to stop band, Hz
numtaps = demod_filter_numtaps          # Size of the FIR filter.
iq_fir_taps = signal.remez(numtaps, [0, cutoff, cutoff + trans_width, 0.5*fs], [1, 0], Hz=fs)
if(0):
    w, h = signal.freqz(iq_fir_taps, [1], worN=2000)
    plot_FIR_response(fs, iq_fir_taps, w, h)           

## Metadata

In [51]:
tx_angles = np.tile(pwi_txAngles, int(np.ceil(rf.shape[1]/len(pwi_txAngles))))*np.pi/180

sequence = PwiSequence(
    pulse=Pulse(center_frequency=pwi_txFreq, n_periods=pwi_nCycles, inverse=False),
    rx_sample_range=(0, rf.shape[1]),
    speed_of_sound=c, # [m/s],
    angles=tx_angles,
    pri=pwi_txPri
)

model=ProbeModel(
    model_id=ProbeModelId("atl", "l7-4"),
    n_elements=probe_elements,
    pitch=probe_pitch,
    curvature_radius=0,
)

probe = ProbeDTO(
    model=model
)

device=Us4RDTO(
    sampling_frequency=fs,
    probe=probe
)

context = arrus.metadata.FrameAcquisitionContext(
    device=device, 
    sequence=sequence,
    raw_sequence=None,
    medium=None,
    custom_data={})

data_desc=EchoDataDescription(
    sampling_frequency=fs,
)

metadata = ConstMetadata(context=context, data_desc=data_desc, input_shape=rf.shape, is_iq_data=False, dtype=np.int16, version=None)

In [52]:
! nvidia-smi

Mon Mar 20 19:58:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8    20W /  N/A |   2429MiB /  5946MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Pipeline: RF Filtering, Down-conversion and Beamforming

In [53]:
# Define the processing pipeline
processing = Pipeline(
    steps=(
        Transpose(axes=(0, 1, 3, 2)),
        #BandpassFilter(),
        FirFilter(taps=rf_fir_taps, num_pkg=None, filter_pkg=None),
        #QuadratureDemodulation(),
        #Decimation(filter_type="fir", filter_coeffs=iq_fir_taps, decimation_factor=1),
        #ReconstructLri(x_grid=x_grid, z_grid=z_grid),
        #Squeeze(),
    ),
    placement="/GPU:0"
)

# Prepare pipeline
processing.prepare(metadata)


deque([<arrus.metadata.ConstMetadata at 0x7fba9c14d520>])

In [63]:
#%%time
#Transfer data to GPU
n = 100
latency = np.zeros(n)
for i in range(n):
    start = time.time()
    din = cp.asarray(rf)
    cp.cuda.Stream.null.synchronize()
    end = time.time()
    latency[i] = 1000*(end-start)

avg = np.mean(latency)
sdev = np.std(latency) 
print('Mean latency [ms]:')
print(avg)
print('Std dev: [ms]:')
print(sdev)

Mean latency [ms]:
23.47891092300415
Std dev: [ms]:
1.2188522953220295


In [60]:
# Run the pipeline
output = processing.process(din)

In [ ]:
%%time
n = 100
start = time.time()
for i in range(n):
    hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)
cp.cuda.Stream.null.synchronize()
end = time.time()
print(f"Average time: {1000*(end-start)/n} [ms]")

In [10]:
lri_data_gpu = output[0]
print(lri_data_gpu.shape)

(150, 200, 250)


In [11]:
! nvidia-smi

Mon Mar 20 19:38:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   50C    P0    30W /  N/A |   2448MiB /  5946MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Angle compounding

In [12]:
# Compounding
AngleCompounder = AngleCompounding(nAngles=len(pwi_txAngles))
AngleCompounder.prepare()

In [45]:
%%time
import time

n = 100
start = time.time()
for i in range(n):
    hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)
cp.cuda.Stream.null.synchronize()
end = time.time()
print(f"Average time: {1000*(end-start)/n} [ms]")

Average time: 0.8315229415893555 [ms]
CPU times: user 84.2 ms, sys: 0 ns, total: 84.2 ms
Wall time: 83.3 ms


In [37]:
hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)

In [32]:
%%timeit -r 1 -n 100
hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)

89.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)


In [ ]:
! nvidia-smi

## B-mode display

In [ ]:
BmodeGenerator = GenerateBmode()
BmodeGenerator.prepare()
Bmode = BmodeGenerator.process(data=hri_data_gpu[5, :, :]) # Done on GPU
BmodeGenerator.plotHistogram(Bmode)
BmodeGenerator.displayBmode(frame=Bmode, dB_range=[40, 80])


## Shear wave detection

In [ ]:
! nvidia-smi

In [ ]:
del lri_data_gpu
del processing
cp._default_memory_pool.free_all_blocks()

In [ ]:
# Shear wave detection
swd_mode              = 'kasai'
swd_zGate_length      = 4 
swd_ensemble_length   = 4

In [ ]:
# Shear wave detection
ShearDetector = ShearwaveDetection(mode=swd_mode, packet_size=swd_ensemble_length, z_gate=swd_zGate_length, frame_pri=pwi_fri, c=c, fc=pwi_txFreq, fs=fs)
ShearDetector.prepare(c=c, frame_pri=200e-6, fs=65e6)

In [ ]:
print(hri_data_gpu.shape)

In [ ]:
#%%timeit -r 10 -n 100
ddata = ShearDetector.process(data=hri_data_gpu)

In [ ]:
print(ddata.shape)

In [ ]:
del hri_data_gpu
cp._default_memory_pool.free_all_blocks()

In [ ]:
# Visualize data
ddata_cpu = ddata.get()
getHistogram(data=ddata_cpu, Vrange = [-2.5e-3, 2.5e-3])
print(ddata_cpu.shape)
DrawShearWaveFrames(data=ddata_cpu, Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])

## Shear wave motion data filtering

In [ ]:
print(ddata.shape)

In [ ]:
# Shear wave motion data filtering in Fourier domain
DirFilter = ShearwaveMotionDataFiltering(sws_range=df_sws_range, f_range=df_f_range, k_range=df_k_range)
DirFilter.prepare(input_shape = ddata.shape, fs=1.0/pwi_fri)

In [ ]:
#%%timeit -r 10 -n 100
ddata_f = DirFilter.process(data=ddata)

In [ ]:
print(ddata_f.shape)

In [ ]:
! nvidia-smi

In [ ]:
# Free GPU memory
del DirFilter
del ShearDetector
del ddata
cp._default_memory_pool.free_all_blocks()

In [ ]:
## Visualize results
# Filter masks
#DirFilter.plotFilterMasks()
# Filtered motion data frames
DrawShearWaveFrames(data=ddata_cpu,    Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
DrawShearWaveFrames(data=ddata_f[0].get(), Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
DrawShearWaveFrames(data=ddata_f[1].get(), Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
# Plot some profiles
#%matplotlib widget
#fig, ax = plt.subplots(1, 1, figsize=(8, 4))
#ax.plot(np.squeeze(ddata_LR[200, 350-15, :]))
#ax.plot(np.squeeze(ddata_LR[200, 350+15, :]))

## Local shear wave speed estimation

In [ ]:
# Overwrite parameters
swse_interp_factor = 5;
swse_interp_order  = 2
swse_d             = 14;
swse_frames        = [0, 99];
swse_SWV_range     = [0.5, 4.0];
swse_x_range       = [[0, 420], [0, 420]]
swse_z_clip        = [5, 10]

In [ ]:
dim = ddata_f.shape
print(dim)
SWS_Estimator = SWS_Estimation(x_range=swse_x_range, z_clip = swse_z_clip, frames_range = swse_frames,
                               d=swse_d, fri = pwi_fri, interp_factor=swse_interp_factor, interp_order=swse_interp_order, 
                                px_pitch=px_size*1e-3, sws_range=swse_SWV_range)
SWS_Estimator.prepare(input_shape = ddata_f.shape)


In [ ]:
#%%timeit -r 1 -n 10
SWV = SWS_Estimator.process(data=ddata_f)

In [ ]:
! nvidia-smi

In [ ]:
#Visualize the results
SWV_cpu = SWV.get()
SWS_cpu = np.squeeze(SWV_cpu[0, ...])
SWS_r_cpu = np.squeeze(SWV_cpu[1, ...])
print()
norm_sws = plt.Normalize(0, 5, True)
norm_r   = plt.Normalize(0, 1, True)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
axs[0,0].imshow(np.squeeze(SWS_cpu[0,:,:]), cmap='jet', norm=norm_sws)
axs[0,0].set_title('SWS_LR [m/s]')

axs[1,0].imshow(np.squeeze(SWS_r_cpu[0,:,:]), cmap='jet', norm=norm_r)
axs[1,0].set_title('r_LR')

axs[0,1].imshow(np.squeeze(SWS_cpu[1,:,:]), cmap='jet', norm=norm_sws)
axs[0,1].set_title('SWS_RL [m/s]')

axs[1,1].imshow(np.squeeze(SWS_r_cpu[1,:,:]), cmap='jet', norm=norm_r)
axs[1,1].set_title('r_RL')


In [ ]:
del SWV
cp._default_memory_pool.free_all_blocks()

In [ ]:
! nvidia-smi

# Post-processing

## Image compounding

In [ ]:
SWS_Compounder = SWS_Compounding()
SWS_Compounder.prepare()

In [ ]:
#%%timeit -r 10 -n 10
SWSc = SWS_Compounder.process(SWV)

In [ ]:
# Visualize results
SWSc_cpu  = SWSc.get()
SWS_cpu = np.squeeze(SWSc_cpu[0, ...])
SWS_r_cpu = np.squeeze(SWSc_cpu[1, ...])
sws_range = [0.5, 4.0]
norm_sws = plt.Normalize(0, 5.0, True)
norm_r   = plt.Normalize(0, 2, True)

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 8))
axs[0].imshow(np.squeeze(SWS_cpu), cmap='jet', norm=norm_sws)
axs[0].set_title('SWS_LR [m/s]')

axs[1].imshow(np.squeeze(SWS_r_cpu), cmap='jet', norm=norm_r)
axs[1].set_title('r_LR')

## Image filtering

In [ ]:
MedianFilter = MedianFiltering(kernel_size=5)
MedianFilter.prepare()

In [ ]:
#%%timeit -r 10 -n 10
SWS_filtered = MedianFilter.process(SWSc)

## Data presentation

In [ ]:
#%%timeit -r 10 -n 10
SWS_filtered_cpu = SWS_filtered[0].get()

In [ ]:
# Visualize SWS map
plotSWSmap(data=SWS_filtered_cpu[:420, swse_d//2:-swse_d//2], px_size=px_size, tick_grid_size=[5, 5], sws_disp_range=[0, 5])